In [1]:
from telegram_models import TelegramExport

chats = TelegramExport.parse_file('result.json').chats

In [4]:
chat = next(chat for chat in chats if chat.name == 'Сергей')
print(chat.name, len(chat.messages), 'messages')

Сергей 45932 messages


In [6]:
import random

my_name = 'Cyber Potato'
my_real_name = 'Денис'

def process_name(name):
    if name == my_name:
        return my_real_name
    return 'Собеседник'

def get_random_chunk(length):
    start = random.randint(0, len(chat.messages) - length)
    return chat.messages[start:start+length]

def get_random_chunk_ensure_text(length):
    while True:
        chunk = get_random_chunk(length)
        if all(message.text for message in chunk):
            return chunk

In [13]:
text = '\n'.join([
    f'{process_name(message.from_)}: {message.text}'
    for message in get_random_chunk_ensure_text(20)
])

print(text)

Денис: пхахахахахахах думаю норм
Денис: дед оценит
Собеседник: Надо лабы спидранить
Денис: Надо!
Собеседник: Я кажется понял чё он хочет в моей лабе
Денис: Ну-ка
Собеседник: Конвертер лонга в стринг
Собеседник: Но я не пойму, он хочет чтобы я по ссылке запись делал, или ретёрнил ответ
Денис: Я думаю по ссылке, но используя String
Собеседник: хм
Собеседник: хм
Собеседник: Какие ты сниппеты для плюсов поставил
Денис: ты имеешь в виду шорткаты типа libsst?
Собеседник: да
Денис: пока никакие
Денис: а ты?
Собеседник: Ты свои поставил?
Собеседник: Я просто видел ты их использовал
Денис: я использовал сниппеты для си
Собеседник: ?


## Summarize using Russian mBART model

Using model: https://huggingface.co/Kirili4ik/mbart_ruDialogSum  
From the developers of [@summarization_bot](https://t.me/summarization_bot)

Possible others:
- https://huggingface.co/IlyaGusev/rugpt3medium_sum_gazeta
- https://huggingface.co/UrukHan/t5-russian-summarization

In [ ]:
# to fix error
%pip uninstall protobuf -y
%pip install protobuf==3.20

In [ ]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

# Download model and tokenizer
model_name = "Kirili4ik/mbart_ruDialogSum"   
tokenizer =  MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)
model.eval()

In [9]:
def summarize(text):
    input_ids = tokenizer(
        [text],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        top_k=0,
        num_beams=3,
        no_repeat_ngram_size=3,
        length_penalty=1.5,
        max_new_tokens=100,
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    return summary

In [14]:
print(summarize(text))

Собеседнику не дают баллов за курсч, но он получил от Дениса 5 баллов. Он не получил матпомощь от матери Дениса, и он даже не сдал бумажку.


## Summarize using Sber ruT5-large model API

Lower quality than Russian mBART model

Homepage: https://cloud.ru/ru/datahub/rugpt3family/summarizer

API reference: https://api.aicloud.sbercloud.ru/public/v2/summarizator/docs

In [ ]:
%pip install ratelimit
%pip install backoff

In [67]:
base_url = 'https://api.aicloud.sbercloud.ru/public/v2/summarizator/predict'

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

In [109]:
from ratelimit import limits, RateLimitException
from backoff import on_exception, expo

import requests


@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=90, period=60)  # just for safety
def summarize_api(text):
    response = requests.post(
        base_url,
        headers=headers,
        json={
            'instances': [
                {
                    "num_beams": 16,
                    "length_penalty": 1.5,
                    'text': text,
                }
            ]
        }
    )

    if response.status_code != 200:
        raise Exception(f'API response: {response.status_code}')

    return response.json()['prediction_best']['bertscore']


In [123]:
print(summarize_api(text))

Собеседник : Конвертер лонга в стринг Денис : я думаю по ссылке, или ретёрнил ответ.


## Tests

In [17]:
text = '\n'.join([
    f'{process_name(message.from_)}: {message.text}'
    for message in get_random_chunk_ensure_text(20)
])

print(text, end='\n\n')
print('Summary:\n' + '-' * 42)
print(summarize(text), end='\n\n')

Денис: Для ноута?
Собеседник: В этих материнках просто во многих есть м2 слоты
Денис: А
Собеседник: м2 это же некст ген ссд
Собеседник: У меня в матери слота нет
Собеседник: Стоит адаптером в pci-e
Денис: Ну здесь нету
Денис: Короче поищу x79-x99 желательно с м2, и камнем подешевле
Денис: И м2 такой же примерно
Денис: Живем
Собеседник: Я не эксперт
Денис: ПХАХАХАХ
Собеседник: Но кажется в комплекте есть проц
Денис: После 100 сообщений с советами
Денис: С матерью?
Денис: Или с м2?
Собеседник: .
Собеседник: Кстати
Собеседник: Я возможно вспомнил прикол
Собеседник: В этих процах нет видеоядра, потому для настройки тебе надо надыбать видюху затычку у Никиты

Summary:
------------------------------------------
Денис ищет проц для ноута. Он ищет x79-x99 желательно с м2, и камнем подешевле.
